# Opis notatnika

Jest to docelowy notatnik, służący przeprowadzaniu eksperymentów rozwiązania. Planowane są następujące porówania:
- SA vs HC
- SA vs RS
- Wpływ wykorzystywanych pokemonów na wyniki
- Wpływ pokemonów legendarnych na wyniki
- Wpływ ilość zmienianych pokemonów w drużynie po przegranej na wyniki
- Ewaluacja nalepszych druzyń (znalezionych przez nasz algorytm)

### Importy

In [1]:
import sys
from pathlib import Path
import pandas as pd

SRC = (Path("..")).resolve()
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))


In [2]:
from classes.pokemon_team import PokemonTeam
from visualization.utils import summarize
from data.data import get_pokemons
from report.reports import PdfReport
from experiments.sa_experiments import run_multiple_runs, compare_to_hill_climb, compare_to_random_search
from visualization.plots import visualize_opponents_typing_distribution, visualize_opponents_stat_sums_violin
from constants import REPORT_DIR

In [3]:
from solvers import SimulatedAnnealingPokemonSolver
from solvers import HillClimbingPokemonSolver
from solvers import RandomSearchPokemonSolver

### Parametry startowe

In [4]:
pokemons = get_pokemons()
all_pokemons = get_pokemons(include_only_final_evolutions=False)
pokemons_and_legendaries = get_pokemons(include_legendary=True, include_only_final_evolutions=True)

In [5]:
print("Ilość dostępnych pokemonów bazowo:", len(pokemons))
print("Ilość dostępnych pokemonów (wszystkie ewolucje):", len(all_pokemons))
print("Ilość dostępnych pokemonów (wszystkie ewolucje + legendarne):", len(pokemons_and_legendaries))

Ilość dostępnych pokemonów bazowo: 359
Ilość dostępnych pokemonów (wszystkie ewolucje): 731
Ilość dostępnych pokemonów (wszystkie ewolucje + legendarne): 427


#### Wielkość przestrzeni przeszukiwań

- Lista oponentów będzie składała się z 50 losowych drużyn, w celu przeprowadzenia szybszych eksperymentów. 
- Następnie najlepsze z drużyn zostaną wziętę pod uwagę w porównaniu i zostaną przetestowane na większym zbiorze, w celu weryfikacji ich skuteczności.
- Każda solver, będzie działał na takim samym budżecie równym 300 iteracją (w celu szybkiego szukania drużyn)
- Z wstępnych eksperymentów wyszło, że duża temperatura negatywnie wpływała na solver SA, więc zaczynać będziemy z wartością 0.3 i zmniejszać będziemy ją o 0.80

In [6]:
OPPONENTS_LIMIT = 50
BUDGET = 300
INITIAL_TEMPERATURE = 0.3
COOLING = 0.8

In [7]:
sa_solver = SimulatedAnnealingPokemonSolver(
    initial_temperature=INITIAL_TEMPERATURE,
    min_temperature=1e-3,
    alpha=COOLING,
    iters_per_temp=30,
    max_evaluations=BUDGET,
    neighbor_replacements=1,
    opponents_limit=OPPONENTS_LIMIT,
    unique_types=True,
    patience=50,
    restarts=0,
    )

hc_solver = HillClimbingPokemonSolver(
    max_evaluations=BUDGET,
    neighbors_per_step=30,
    neighbor_replacements=1,
    opponents_limit=OPPONENTS_LIMIT,
    unique_types=True,
    patience=50,
    restarts=0)

rs_solver = RandomSearchPokemonSolver(trials=BUDGET, opponents_limit=OPPONENTS_LIMIT, unique_types=True)

### Opponents (50 drużyn)

In [8]:
pokemons = get_pokemons()
opponents = PokemonTeam.generate_unique_teams(
    pokemons,
    opponents_limit=OPPONENTS_LIMIT,
    max_attempts=OPPONENTS_LIMIT * 3,
    team_size=6,
    unique_types=True,
)

# Porównanie SA vs HC (10 runów)

In [9]:
opponents_typings = visualize_opponents_typing_distribution(opponents)
opponents_stats = visualize_opponents_stat_sums_violin(opponents)

df_sa_vs_hc = compare_to_hill_climb(pokemons, sa_solver, hc_solver, opponents=opponents, runs=10)

sum_sa_hc = summarize(df_sa_vs_hc)

report = PdfReport(str(REPORT_DIR / "sa_vs_hc_last_evolutions.pdf"), title="SA vs HC (same opponents)")

report.add_text(
    f"""Setup
- opponents_limit = {len(opponents)}
- runs = 10
- budget = {BUDGET}
- SA: initial_temperature = {INITIAL_TEMPERATURE}, cooling = {COOLING}
- restarts = 0
- patience = 50
- same opponents for all experiments
"""
)

report.add_figure(opponents_typings)
report.add_figure(opponents_stats)


report.add_dataframe(df_sa_vs_hc, "SA vs HC (same start per run): per-run results")
report.add_dataframe(sum_sa_hc, "SA vs HC: summary stats")


report.write()
print(f"Saved: {REPORT_DIR / 'report_sa_experiments.pdf'}")

C:\Users\barte\OneDrive\Pulpit\POP - projekt\pop-project\src\visualization\plots.py:53: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(


Comparing SA solver to Hill Climbing solver...
 Run 1/10...
  SA fitness: 0.2415944540727903, HC fitness: 0.49635087719298243
  SA team: ['Bellossom', 'Porygon-Z', 'Alomomola', 'Lucario', 'Donphan', 'Hydreigon']
  HC team: ['Blissey', 'Emboar', 'Stunfisk', 'Vaporeon', 'Drifblim', 'Aromatisse']
 Run 2/10...
  SA fitness: 0.2625649350649351, HC fitness: 0.46631578947368413
  SA team: ['Wobbuffet', 'Clawitzer', 'Mamoswine', 'Eelektross', 'Pidgeot', 'Kricketune']
  HC team: ['Metagross', 'Abomasnow', 'Slaking', 'Heracross', 'Mudsdale', 'Alomomola']
 Run 3/10...
  SA fitness: 0.22547445255474455, HC fitness: 0.5534953703703703
  SA team: ['Hariyama', 'Tyranitar', 'Dragalge', 'Comfey', 'Audino', 'Trevenant']
  HC team: ['Mudsdale', 'Gigalith', 'Abomasnow', 'Hariyama', 'Blissey', 'Wobbuffet']
 Run 4/10...
  SA fitness: 0.2174814814814815, HC fitness: 0.5040772014475272
  SA team: ['Lickilicky', 'Golurk', 'Aromatisse', 'Blaziken', 'Archeops', 'Hypno']
  HC team: ['Tyranitar', 'Blissey', 'Stunf

# Porównanie SA vs RS

In [10]:
opponents_typings = visualize_opponents_typing_distribution(opponents)
opponents_stats = visualize_opponents_stat_sums_violin(opponents)

df_sa_vs_rs = compare_to_random_search(pokemons, sa_solver, rs_solver, opponents=opponents, runs=10)

sum_sa_rs = summarize(df_sa_vs_rs)

report = PdfReport(str(REPORT_DIR / "sa_vs_rs_last_evolutions.pdf"), title="SA vs RS (same opponents)")
report.add_text(
    f"""Setup
- opponents_limit = {len(opponents)}
- runs = 10
- budget = {BUDGET}
- SA: initial_temperature = {INITIAL_TEMPERATURE}, cooling = {COOLING}
- restarts = 0
- patience = 50
- same opponents for all experiments
"""
)

report.add_figure(opponents_typings)
report.add_figure(opponents_stats)


report.add_dataframe(df_sa_vs_rs, "SA vs RS (same start per run): per-run results")
report.add_dataframe(sum_sa_rs, "SA vs RS: summary stats")


report.write()

C:\Users\barte\OneDrive\Pulpit\POP - projekt\pop-project\src\visualization\plots.py:53: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(


Comparing SA solver to Random Search solver...
 Run 1/10...
  SA fitness: 0.24778378378378377, RS fitness: 0.3916196136701337
  SA team: ['Avalugg', 'Rhyperior', 'Granbull', 'Floatzel', 'Maractus', 'Bouffalant']
  RS team: ['Snorlax', 'Rhyperior', 'Sunflora', 'Durant', 'Oricorio', 'Wobbuffet']
 Run 2/10...
  SA fitness: 0.3596501457725948, RS fitness: 0.3995862068965517
  SA team: ['Swampert', 'Gogoat', 'Gumshoos', 'Throh', 'Wobbuffet', 'Glaceon']
  RS team: ['Blissey', 'Krookodile', 'Conkeldurr', 'Forretress', 'Walrein', 'Heatmor']
 Run 3/10...
  SA fitness: 0.1826843100189036, RS fitness: 0.32974276527331187
  SA team: ['Umbreon', 'Sunflora', 'Castform', 'Flygon', 'Beedrill', 'Hariyama']
  RS team: ['Pinsir', 'Dhelmise', 'Greninja', 'Mudsdale', 'Turtonator', 'Blissey']
 Run 4/10...
  SA fitness: 0.22892921960072596, RS fitness: 0.34444444444444444
  SA team: ['Yanmega', 'Lucario', 'Umbreon', 'Dunsparce', 'Mudsdale', 'Tangrowth']
  RS team: ['Torterra', 'Passimian', 'Turtonator', 'Wea

# Uwzględnienie pokemonów legendarnych (tylko SA)

In [9]:
OPPONENTS_LIMIT = 100

opponents = PokemonTeam.generate_unique_teams(
    pokemons_and_legendaries,
    opponents_limit=OPPONENTS_LIMIT,
    max_attempts=OPPONENTS_LIMIT * 3,
    team_size=6,
    unique_types=True,
)

sa_solver_legendaries = SimulatedAnnealingPokemonSolver(
    initial_temperature=0.1,
    min_temperature=1e-3,
    alpha=0.5,
    iters_per_temp=10,
    max_evaluations=BUDGET,
    neighbor_replacements=1,
    opponents_limit=OPPONENTS_LIMIT,
    unique_types=True,
    patience=50,
    restarts=3,
    )

In [10]:
opponents_typings = visualize_opponents_typing_distribution(opponents)
opponents_stats = visualize_opponents_stat_sums_violin(opponents)

df_sa = run_multiple_runs(pokemons_and_legendaries, sa_solver_legendaries, opponents=opponents, runs=10)

sum_sa = summarize(df_sa)

report = PdfReport(str(REPORT_DIR / "sa_with_legendaries.pdf"), title="SA including legendary pokemons")

report.add_text(
    f"""Setup
- opponents_limit = {len(opponents)}
- runs = 10
- budget = {BUDGET}
- SA: initial_temperature = {0.1}, cooling = {0.5}
- restarts = 3
- patience = 50
"""
)

report.add_figure(opponents_typings)
report.add_figure(opponents_stats)


report.add_dataframe(df_sa, "SA with legendaries")
report.add_dataframe(sum_sa, "SA: summary stats")


report.write()

C:\Users\barte\OneDrive\Pulpit\POP - projekt\pop-project\src\visualization\plots.py:53: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(


Running 10 runs of the SA solver...
 Run 1/10...
  Best fitness: 0.3791472868217054
  Team: ['Lunala', 'Blastoise', 'Groudon', 'Braviary', 'Nihilego', 'Throh']
 Run 2/10...
  Best fitness: 0.42151866151866146
  Team: ['Blissey', 'Gyarados', 'Zekrom', 'Nidoking', 'Throh', 'Xerneas']
 Run 3/10...
  Best fitness: 0.37330645161290327
  Team: ['Tyranitar', 'Mewtwo', 'Zekrom', 'Whiscash', 'Tangrowth', 'Exploud']
 Run 4/10...
  Best fitness: 0.4248113207547169
  Team: ['Wailord', 'Tangrowth', 'Aurorus', 'Mewtwo', 'Guzzlord', 'Xerneas']
 Run 5/10...
  Best fitness: 0.42966625463535224
  Team: ['Blissey', 'Necrozma', 'Blastoise', 'Kyurem', 'Stunfisk', 'Hariyama']
 Run 6/10...
  Best fitness: 0.45223577235772355
  Team: ['Buzzwole', 'Nihilego', 'Mewtwo', 'Glalie', 'Zygarde', 'Arceus']
 Run 7/10...
  Best fitness: 0.40495778045838354
  Team: ['Crabominable', 'Gogoat', 'Wobbuffet', 'Snorlax', 'Nihilego', 'Drifblim']
 Run 8/10...
  Best fitness: 0.4124199743918054
  Team: ['Blissey', 'Aromatisse', 

# Wpływ ilości zmienianych pokemonów po przegranej - dla różnych drużyn (tylko SA)

In [21]:
OPPONENTS_LIMIT = 200
BUDGET = 300
INITIAL_TEMPERATURE = 0.2
COOLING = 0.8
ITERS_PER_TEMP = 5

In [22]:
opponents = PokemonTeam.generate_unique_teams(
    pokemons,
    opponents_limit=OPPONENTS_LIMIT,
    max_attempts=OPPONENTS_LIMIT * 3,
    team_size=6,
    unique_types=True,
)


for neighbour_replacement in [1, 2, 3, 4]:

    opponents_typings = visualize_opponents_typing_distribution(opponents)
    opponents_stats = visualize_opponents_stat_sums_violin(opponents)

    sa_solver_different_replacements = SimulatedAnnealingPokemonSolver(
        initial_temperature=INITIAL_TEMPERATURE,
        min_temperature=1e-3,
        alpha=COOLING,
        iters_per_temp=ITERS_PER_TEMP,
        max_evaluations=BUDGET,
        neighbor_replacements=neighbour_replacement,
        opponents_limit=OPPONENTS_LIMIT,
        unique_types=True,
        patience=50,
        restarts=0,
        )


    df_sa = run_multiple_runs(pokemons, sa_solver_different_replacements, opponents=opponents, runs=8)
    sum_sa = summarize(df_sa)

    report = PdfReport(str(REPORT_DIR / f"sa_experiment_neighbours_{neighbour_replacement}.pdf"), title="SA with different neighbor replacements")

    report.add_text(
        f"""Setup
    - opponents_limit = {len(opponents)}
    - runs = 8
    - budget = {BUDGET}
    - SA: initial_temperature = {INITIAL_TEMPERATURE}, cooling = {COOLING}
    - restarts = 0
    - patience = 50
    - legendaries = False
    - neighbor_replacements = {neighbour_replacement}
    - iters_per_temp = {ITERS_PER_TEMP}
    """
    )

    report.add_figure(opponents_typings)
    report.add_figure(opponents_stats)


    report.add_dataframe(df_sa, "SA experiment")
    report.add_dataframe(sum_sa, "SA: summary stats")


    report.write()

C:\Users\barte\OneDrive\Pulpit\POP - projekt\pop-project\src\visualization\plots.py:53: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(


Running 8 runs of the SA solver...
 Run 1/8...
  Best fitness: 0.4659675480769231
  Team: ['Musharna', 'Garchomp', 'Blissey', 'Aurorus', 'Throh', 'Mandibuzz']
 Run 2/8...
  Best fitness: 0.4374
  Team: ['Mudsdale', 'Blissey', 'Tangrowth', 'Reuniclus', 'Emboar', 'Drifblim']
 Run 3/8...
  Best fitness: 0.4040554821664465
  Team: ['Wobbuffet', 'Rhyperior', 'Charizard', 'Venusaur', 'Hariyama', 'Slaking']
 Run 4/8...
  Best fitness: 0.3076333333333333
  Team: ['Houndoom', 'Lickilicky', 'Chesnaught', 'Aurorus', 'Stunfisk', 'Samurott']
 Run 5/8...
  Best fitness: 0.3551324965132497
  Team: ['Musharna', 'Golurk', 'Alomomola', 'Tyranitar', 'Hariyama', 'Audino']
 Run 6/8...
  Best fitness: 0.251828665568369
  Team: ['Glalie', 'Hypno', 'Rampardos', 'Garbodor', 'Snorlax', 'Seismitoad']
 Run 7/8...
  Best fitness: 0.4824135802469135
  Team: ['Abomasnow', 'Blaziken', 'Blissey', 'Drifblim', 'Lanturn', 'Excadrill']
 Run 8/8...
  Best fitness: 0.37328038674033154
  Team: ['Archeops', 'Meganium', 'Bliss

C:\Users\barte\OneDrive\Pulpit\POP - projekt\pop-project\src\visualization\plots.py:53: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(


Running 8 runs of the SA solver...
 Run 1/8...
  Best fitness: 0.3296694214876033
  Team: ['Blaziken', 'Golem', 'Metagross', 'Tangrowth', 'Beartic', 'Wailord']
 Run 2/8...
  Best fitness: 0.35872819767441866
  Team: ['Slaking', 'Salamence', 'Slurpuff', 'Vaporeon', 'Musharna', 'Rhyperior']
 Run 3/8...
  Best fitness: 0.31301639344262294
  Team: ['Mandibuzz', 'Kingdra', 'Rhyperior', 'Sceptile', 'Heracross', 'Snorlax']
 Run 4/8...
  Best fitness: 0.442280912364946
  Team: ['Blissey', 'Rhyperior', 'Wobbuffet', 'Lapras', 'Florges', 'Bisharp']
 Run 5/8...
  Best fitness: 0.31808843537414966
  Team: ['Excadrill', 'Garbodor', 'Mandibuzz', 'Wobbuffet', 'Sylveon', 'Slaking']
 Run 6/8...
  Best fitness: 0.317915512465374
  Team: ['Crabominable', 'Wobbuffet', 'Heatmor', 'Snorlax', 'Granbull', 'Relicanth']
 Run 7/8...
  Best fitness: 0.2853125
  Team: ['Beheeyem', 'Bewear', 'Banette', 'Aurorus', 'Swampert', 'Mandibuzz']
 Run 8/8...
  Best fitness: 0.31887303851640514
  Team: ['Gastrodon', 'Meganium

C:\Users\barte\OneDrive\Pulpit\POP - projekt\pop-project\src\visualization\plots.py:53: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(


Running 8 runs of the SA solver...
 Run 1/8...
  Best fitness: 0.345
  Team: ['Pangoro', 'Blastoise', 'Meowstic', 'Blissey', 'Meganium', 'Garchomp']
 Run 2/8...
  Best fitness: 0.39146341463414636
  Team: ['Throh', 'Clefable', 'Magmortar', 'Blissey', 'Wobbuffet', 'Noivern']
 Run 3/8...
  Best fitness: 0.4340356265356266
  Team: ['Blissey', 'Kommo-o', 'Amoonguss', 'Whiscash', 'Masquerain', 'Wobbuffet']
 Run 4/8...
  Best fitness: 0.36182142857142857
  Team: ['Krookodile', 'Magmortar', 'Blissey', 'Relicanth', 'Leavanny', 'Swalot']
 Run 5/8...
  Best fitness: 0.32113385826771645
  Team: ['Golduck', 'Mothim', 'Rhyperior', 'Simisage', 'Kangaskhan', 'Wobbuffet']
 Run 6/8...
  Best fitness: 0.40561789772727275
  Team: ['Rhyperior', 'Blissey', 'Abomasnow', 'Crawdaunt', 'Aromatisse', 'Vivillon']
 Run 7/8...
  Best fitness: 0.4152828947368421
  Team: ['Metagross', 'Blastoise', 'Blissey', 'Mudsdale', 'Gogoat', 'Aurorus']
 Run 8/8...
  Best fitness: 0.23525876460767947
  Team: ['Golurk', 'Gorebyss

C:\Users\barte\OneDrive\Pulpit\POP - projekt\pop-project\src\visualization\plots.py:53: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(


Running 8 runs of the SA solver...
 Run 1/8...
  Best fitness: 0.3355034482758621
  Team: ['Clefable', 'Electrode', 'Slaking', 'Accelgor', 'Wobbuffet', 'Drifblim']
 Run 2/8...
  Best fitness: 0.3702232142857143
  Team: ['Snorlax', 'Golurk', 'Sylveon', 'Alomomola', 'Volcarona', 'Wobbuffet']
 Run 3/8...
  Best fitness: 0.3173380281690141
  Team: ['Sawk', 'Drapion', 'Blissey', 'Vaporeon', 'Heatmor', 'Exeggutor']
 Run 4/8...
  Best fitness: 0.37647208121827413
  Team: ['Skarmory', 'Hariyama', 'Vaporeon', 'Cradily', 'Garchomp', 'Blissey']
 Run 5/8...
  Best fitness: 0.36723308270676697
  Team: ['Kommo-o', 'Tangrowth', 'Flareon', 'Sharpedo', 'Mudsdale', 'Blissey']
 Run 6/8...
  Best fitness: 0.3411007194244604
  Team: ['Goodra', 'Conkeldurr', 'Serperior', 'Butterfree', 'Blissey', 'Whiscash']
 Run 7/8...
  Best fitness: 0.41715950920245404
  Team: ['Dhelmise', 'Blissey', 'Gyarados', 'Wobbuffet', 'Machamp', 'Rhyperior']
 Run 8/8...
  Best fitness: 0.24911564625850338
  Team: ['Slaking', 'Treve

# GRID SEARCH

In [18]:
OPPONENTS_LIMIT = 30
BUDGET = 100
INITIAL_TEMPERATURE = 0.3
COOLING = 0.8
ITERS_PER_TEMP = 4

In [19]:
GRID_SEARCH_PARAMS = {
    'neighbor_replacements': [1, 2, 3],
    'initial_temperature': [0.1, 0.3],
    'cooling': [0.8, 0.9],
    'legendaries': [False, True],
}

In [20]:
from sklearn.model_selection import ParameterGrid

for index, params in enumerate(ParameterGrid(GRID_SEARCH_PARAMS)):
    cooling = params['cooling']
    initial_temperature = params['initial_temperature']
    legendaries = params['legendaries']
    neighbour_replacements = params['neighbor_replacements']
    print(f"Running experiment {index + 1}/{len(ParameterGrid(GRID_SEARCH_PARAMS))} with params: {params}")

    opponents_normal = PokemonTeam.generate_unique_teams(
        pokemons,
        opponents_limit=OPPONENTS_LIMIT,
        max_attempts=OPPONENTS_LIMIT * 3,
        team_size=6,
        unique_types=True,
    )

    opponents_legendaries = PokemonTeam.generate_unique_teams(
        pokemons_and_legendaries,
        opponents_limit=OPPONENTS_LIMIT,
        max_attempts=OPPONENTS_LIMIT * 3,
        team_size=6,
        unique_types=True,
    )

    if legendaries:
        opponents = opponents_legendaries
        pokemon_to_use = pokemons_and_legendaries
    else:
        opponents = opponents_normal
        pokemon_to_use = pokemons

    opponents_typings = visualize_opponents_typing_distribution(opponents)
    opponents_stats = visualize_opponents_stat_sums_violin(opponents)

    sa_solver_different_replacements = SimulatedAnnealingPokemonSolver(
        initial_temperature=initial_temperature,
        min_temperature=1e-3,
        alpha=cooling,
        iters_per_temp=ITERS_PER_TEMP,
        max_evaluations=BUDGET,
        neighbor_replacements=neighbour_replacements,
        opponents_limit=OPPONENTS_LIMIT,
        unique_types=True,
        patience=50,
        restarts=0,
        )


    df_sa = run_multiple_runs(pokemon_to_use, sa_solver_different_replacements, opponents=opponents, runs=8)
    sum_sa = summarize(df_sa)

    report = PdfReport(str(REPORT_DIR / f"sa_experiment_{index + 1}_legendaries_{legendaries}_cooling_{cooling}_temp_{initial_temperature}_neigbours_{neighbour_replacements}.pdf"), title="SA with different hyperparameters")

    report.add_text(
        f"""Setup
    - opponents_limit = {len(opponents)}
    - runs = 8
    - budget = {BUDGET}
    - SA: initial_temperature = {initial_temperature}, cooling = {cooling}
    - restarts = 0
    - patience = 50
    - legendaries = {legendaries}
    - neighbor_replacements = {neighbour_replacements}
    - iters_per_temp = {ITERS_PER_TEMP}
    """
    )

    report.add_figure(opponents_typings)
    report.add_figure(opponents_stats)


    report.add_dataframe(df_sa, "SA experiment")
    report.add_dataframe(sum_sa, "SA: summary stats")


    report.write()

Running experiment 1/24 with params: {'cooling': 0.8, 'initial_temperature': 0.1, 'legendaries': False, 'neighbor_replacements': 1}


C:\Users\barte\OneDrive\Pulpit\POP - projekt\pop-project\src\visualization\plots.py:53: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(


Running 8 runs of the SA solver...
 Run 1/8...
  Best fitness: 0.3820754716981132
  Team: ['Rampardos', 'Arcanine', 'Nidoking', 'Chimecho', 'Hariyama', 'Blissey']
 Run 2/8...
  Best fitness: 0.3824441404468764
  Team: ['Gigalith', 'Amoonguss', 'Vaporeon', 'Exploud', 'Wobbuffet', 'Garchomp']
 Run 3/8...
  Best fitness: 0.3138670852956567
  Team: ['Musharna', 'Arcanine', 'Goodra', 'Aurorus', 'Silvally', 'Gogoat']
 Run 4/8...
  Best fitness: 0.33363453815261046
  Team: ['Slaking', 'Banette', 'Grumpig', 'Rhyperior', 'Alomomola', 'Abomasnow']
 Run 5/8...
  Best fitness: 0.2346516007532957
  Team: ['Krookodile', 'Salamence', 'Infernape', 'Miltank', 'Venusaur', 'Lunatone']
 Run 6/8...
  Best fitness: 0.26039279869067106
  Team: ['Camerupt', 'Hypno', 'Dunsparce', 'Toxicroak', 'Drifblim', 'Aurorus']
 Run 7/8...
  Best fitness: 0.3733525733525733
  Team: ['Chimecho', 'Rampardos', 'Haxorus', 'Mudsdale', 'Blissey', 'Abomasnow']
 Run 8/8...
  Best fitness: 0.4320700896495518
  Team: ['Musharna', 'P

C:\Users\barte\OneDrive\Pulpit\POP - projekt\pop-project\src\visualization\plots.py:53: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(


Running 8 runs of the SA solver...
 Run 1/8...
  Best fitness: 0.40874074074074074
  Team: ['Salamence', 'Abomasnow', 'Tyranitar', 'Wobbuffet', 'Excadrill', 'Arcanine']
 Run 2/8...
  Best fitness: 0.33127771060056427
  Team: ['Blissey', 'Wobbuffet', 'Gyarados', 'Passimian', 'Arcanine', 'Rampardos']
 Run 3/8...
  Best fitness: 0.274910941475827
  Team: ['Excadrill', 'Skuntank', 'Reuniclus', 'Crabominable', 'Snorlax', 'Electivire']
 Run 4/8...
  Best fitness: 0.27774327122153214
  Team: ['Avalugg', 'Tangrowth', 'Musharna', 'Hippowdon', 'Braviary', 'Lanturn']
 Run 5/8...
  Best fitness: 0.4108955223880597
  Team: ['Feraligatr', 'Eelektross', 'Tyranitar', 'Dragalge', 'Glalie', 'Blissey']
 Run 6/8...
  Best fitness: 0.33873873873873883
  Team: ['Clefable', 'Nidoqueen', 'Altaria', 'Aurorus', 'Wobbuffet', 'Vaporeon']
 Run 7/8...
  Best fitness: 0.4145598194130925
  Team: ['Jellicent', 'Garchomp', 'Mandibuzz', 'Wobbuffet', 'Blissey', 'Gogoat']
 Run 8/8...
  Best fitness: 0.29319286871961103
  

...

Running 8 runs of the SA solver...
 Run 1/8...
  Best fitness: 0.3262312078797304
  Team: ['Stoutland', 'Dhelmise', 'Pinsir', 'Zygarde', 'Terrakion', 'Musharna']
 Run 2/8...
  Best fitness: 0.23046070460704604
  Team: ['Torkoal', 'Krookodile', 'Milotic', 'Kyurem', 'Reuniclus', 'Throh']
 Run 3/8...
  Best fitness: 0.29685534591194973
  Team: ['Snorlax', 'Nihilego', 'Vaporeon', 'Guzzlord', 'Espeon', 'Hippowdon']
 Run 4/8...
  Best fitness: 0.21611842105263157
  Team: ['Honchkrow', 'Kabutops', 'Virizion', 'Golurk', 'Drampa', 'Wobbuffet']
 Run 5/8...
  Best fitness: 0.25817091454272867
  Team: ['Gothitelle', 'Celesteela', 'Krookodile', 'Blissey', 'Venomoth', 'Regirock']
 Run 6/8...
  Best fitness: 0.3307855626326964
  Team: ['Rayquaza', 'Slaking', 'Mudsdale', 'Gengar', 'Buzzwole', 'Mewtwo']
 Run 7/8...
  Best fitness: 0.26596091205211725
  Team: ['Mewtwo', 'Swalot', 'Ho-Oh', 'Dedenne', 'Alomomola', 'Aggron']
 Run 8/8...
  Best fitness: 0.32274509803921564
  Team: ['Abomasnow', 'Cresselia',

C:\Users\barte\OneDrive\Pulpit\POP - projekt\pop-project\src\visualization\plots.py:53: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(


Running 8 runs of the SA solver...
 Run 1/8...
  Best fitness: 0.3494725738396624
  Team: ['Xerneas', 'Rhyperior', 'Vaporeon', 'Simisear', 'Roserade', 'Yveltal']
 Run 2/8...
  Best fitness: 0.31447437929212885
  Team: ['Armaldo', 'Zapdos', 'Lucario', 'Regigigas', 'Zygarde', 'Sharpedo']
 Run 3/8...
  Best fitness: 0.2936708860759494
  Team: ['Victini', 'Dialga', 'Groudon', 'Conkeldurr', 'Crawdaunt', 'Eelektross']
 Run 4/8...
  Best fitness: 0.3735244519392917
  Team: ['Groudon', 'Ambipom', 'Buzzwole', 'Trevenant', 'Swalot', 'Yveltal']
 Run 5/8...
  Best fitness: 0.4091755319148936
  Team: ['Walrein', 'Exploud', 'Bellossom', 'Zygarde', 'Necrozma', 'Drifblim']
 Run 6/8...
  Best fitness: 0.3625407166123778
  Team: ['Lickilicky', 'Nidoking', 'Kyurem', 'Yveltal', 'Bellossom', 'Necrozma']
 Run 7/8...
  Best fitness: 0.4320206282231599
  Team: ['Heatran', 'Hitmonlee', 'Groudon', 'Glaceon', 'Blissey', 'Drifblim']
 Run 8/8...
  Best fitness: 0.41238938053097346
  Team: ['Mudsdale', 'Blissey', '

C:\Users\barte\OneDrive\Pulpit\POP - projekt\pop-project\src\visualization\plots.py:53: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  sns.violinplot(


Running 8 runs of the SA solver...
 Run 1/8...
  Best fitness: 0.23991507430997883
  Team: ['Zygarde', 'Conkeldurr', 'Registeel', 'Furret', 'Rampardos', 'Dusknoir']
 Run 2/8...
  Best fitness: 0.33156089193825034
  Team: ['Diancie', 'Regigigas', 'Venomoth', 'Groudon', 'Vaporeon', 'Gogoat']
 Run 3/8...
  Best fitness: 0.3753687315634219
  Team: ['Decidueye', 'Groudon', 'Hypno', 'Weezing', 'Blissey', 'Umbreon']
 Run 4/8...
  Best fitness: 0.41518518518518516
  Team: ['Groudon', 'Tangrowth', 'Lunala', 'Guzzlord', 'Escavalier', 'Raikou']
 Run 5/8...
  Best fitness: 0.30288520564763655
  Team: ['Dewgong', 'Mewtwo', 'Rhyperior', 'Bouffalant', 'Vikavolt', 'Klinklang']
 Run 6/8...
  Best fitness: 0.30170940170940175
  Team: ['Arceus', 'Abomasnow', 'Conkeldurr', 'Latios', 'Donphan', 'Kyogre']
 Run 7/8...
  Best fitness: 0.4186147186147186
  Team: ['Lurantis', 'Solrock', 'Groudon', 'Kyogre', 'Blissey', 'Charizard']
 Run 8/8...
  Best fitness: 0.31227373068432673
  Team: ['Reshiram', 'Blissey', '